<a href="https://colab.research.google.com/github/bigirimanainnocent12/Traitement_du_fichier_text/blob/main/Traitement_du_fichier_texte.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [38]:
from datetime import date
import os
import pandas as pd
import time
import numpy as np
from tqdm import tqdm
from rich import print

def traiter_fichiers(dossier_fichiers, dossier_sauvegarde, dossier_fermenteur):
    # Mesurer le temps de début
    start_time = time.time()

    # Liste des encodages possibles
    encodages = ['utf-8', 'latin-1', 'cp1252', 'ascii', 'utf-16', 'utf-32']

    # Créer des dossiers pour sauvegarder les fichiers Excel
    os.makedirs(dossier_sauvegarde, exist_ok=True)
    os.makedirs(dossier_fermenteur, exist_ok=True)  # Créer le dossier Fermenteur

    # Obtenir la liste des fichiers texte dans le dossier
    fichiers_texte = os.listdir(dossier_fichiers)

    # Parcourir tous les fichiers texte dans le dossier avec une barre de progression
    for fichier_texte in tqdm(fichiers_texte, desc="Traitement des fichiers"):
        chemin_fichier = os.path.join(dossier_fichiers, fichier_texte)

        for encodage in encodages:
            try:
                with open(chemin_fichier, "r", encoding=encodage) as fichier:
                    contenu = fichier.read()
                print(f"Fichier '{fichier_texte}' lu avec l'encodage : {encodage}")

                df = pd.DataFrame([x.split('\t') for x in contenu.split('\n') if x])
                Fermenteur = df.loc[:, 0][1].split('=')[1]
                date_experience = df.loc[:, 0][df[df[0].str.startswith('Début')].index[0]].split('=')[1]
                Souche = df.loc[:, 0][df[df[0].str.startswith('Souche')].index[0]].split('=')[1]
                Milieu = df.loc[:, 0][df[df[0].str.startswith('Milieu')].index[0]].split('=')[1]
                volume = "Salle"
                dicti = {'Fermenteur': Fermenteur, 'date_experience': date_experience, 'Souche': Souche, 'Milieu': Milieu, 'volume': volume}
                data = pd.DataFrame(dicti, index=[0])
                data['date_experience'] = pd.to_datetime(data['date_experience'], format='%d/%m/%Y %H:%M')
                data['Code'] = 'F' + data['Fermenteur'].astype(str) + '_' + data['date_experience'].astype(str)

                valeurs_a_chercher = ["[Données]", "[Donn�es]"]
                df = df.loc[:,][df.loc[df.loc[:, 0].isin(valeurs_a_chercher)].index[0] + 1:]
                nouvelles_colonnes = df.iloc[0, :len(df.columns)].tolist()
                df.columns = nouvelles_colonnes
                # Supprimer la première ligne (qui a été utilisée comme noms de colonnes)
                df = df.iloc[1:]








                # Réinitialiser l'index
                df = df.reset_index(drop=True)
                df.drop(columns=['T°C', 'Poids', 'Volume', 'FinPhas', 'Vinst', 'Csg V', 'V5', 'Acc G','Tps inject.'], inplace=True)

                df['Code']=data['Code'][0]
                df.rename(columns={'Csg T': 'Csg_T'}, inplace=True)
                df['Temps'] = df['Temps'].str.replace(',', '.').astype(float)
                df['Csg_T'] = df['Csg_T'].str.replace(',', '.').astype(float)
                df['CO2']=df['CO2'].str.replace(',', '.').astype(float)
                df['V11']=df['V11'].str.replace(',', '.').astype(float)
                # Traitement des ecarts entre deux observations de la colonne temps
                taille=len(df)
                compteur=0
                for i in np.arange(taille-1):
                  z=df['Temps'].iloc[i+1]-df['Temps'].iloc[i]
                  if z>70:
                     compteur=compteur+1


                # Traitement des valeurs manquantes
                S = 0
                for i in np.arange(len(df)):
                    if df['CO2'].isna().iloc[i] == True or df['V11'].isna().iloc[i] == True or df['Csg_T'].isna().iloc[i] == True:
                        S = S + 1




                if S==0 and compteur==0 and len(df) > 100 and df['V11'].iloc[-1]<0.1 and df['CO2'].max()>60 and df['V11'].iloc[12]<0.2 :
                # Chemin complet pour sauvegarder le fichier Excel

                  chemin_excel = os.path.join(dossier_sauvegarde, f"{os.path.splitext(fichier_texte)[0]}.xlsx")
                  df.to_excel(chemin_excel, index=False)
                  chemin_fermenteur = os.path.join(dossier_fermenteur, f"{os.path.splitext(fichier_texte)[0]}.xlsx")
                  data.to_excel(chemin_fermenteur, index=False)

                  print(f"Les DataFrames ont été sauvegardés sous format Excel dans '{chemin_excel}' et '{chemin_fermenteur}'.")

                else:
                  print("[Le fichier ne respecte pas toutes les conditions requises pour être sauvegarder")

                break  # Sortir de la boucle si la lecture est réussie

            except UnicodeDecodeError:
                print(f"Échec de la lecture avec l'encodage : {encodage}")
            except FileNotFoundError:
                print(f"Erreur : Le fichier '{chemin_fichier}' n'existe pas.")
                break
            except Exception as e:
                print(f"Une erreur s'est produite avec l'encodage {encodage} : {e}")

    # Mesurer le temps de fin
    end_time = time.time()
    print(f"Temps total de traitement : {end_time - start_time} secondes")

# Appeler la fonction avec les chemins appropriés
traiter_fichiers("/content/drive/MyDrive/code", "/content/drive/MyDrive/donnees", "/content/drive/MyDrive/Fermenteur")

Traitement des fichiers:   0%|          | 0/5 [00:00<?, ?it/s]

Échec de la lecture avec l'encodage : utf-8

Fichier 'F1_08022021_1817_Brioso_CfPSM_T200_NV_SYNC.txt' lu avec l'encodage : latin-1

[Le fichier ne respecte pas toutes les conditions requises pour être sauvegarder

Échec de la lecture avec l'encodage : utf-8

...

Échec de la lecture avec l'encodage : utf-8

Fichier 'F1_07092021_1228_Laktia_MSN250S230FA2_T240_NV_SYNC.txt' lu avec l'encodage : latin-1

[Le fichier ne respecte pas toutes les conditions requises pour être sauvegarder

Traitement des fichiers: 100%|██████████| 5/5 [00:01<00:00,  3.59it/s]


Temps total de traitement : 1.397550106048584 secondes